In [1]:
# read the file 'dataset_reg_pair_filled.csv' and generate synthetic data
# first read the file, then make a list of the source training images
# then for each image, generate 10 synthetic images with random affine transformation

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random
import csv
from PIL import Image
import shutil

import torch

from utils.utils0 import tensor_affine_transform, transform_to_displacement_field
from utils.utils1 import transform_points_DVF
from utils.SuperPoint import SuperPointFrontend, PointTracker
superpoint = SuperPointFrontend('utils/superpoint_v1.pth', nms_dist=4,
                          conf_thresh=0.015, nn_thresh=0.7, cuda=True)

# read the file
df = pd.read_csv('Dataset/dataset_reg_pair_filled.csv')
print(df.shape)
df.head()

Device: cuda
(5040, 9)


,source,Source ROI,target,Target ROI,training,Warped target images,Warped target ROI,Execution time,Directory
0,2011248_20161215__L_b2.jpg,NaN,2011248_20161215__L_b1.jpg,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lb
1,2011248_20161215__L_b2.jpg,NaN,2011248_20161215__L_b3.jpg,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lb
2,2011248_20161215__L_b1.jpg,NaN,2011248_20161215__L_b3.jpg,NaN,1,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lb
3,2011248_20161215__L_c2.jpg,NaN,2011248_20161215__L_c1.jpg,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lc
4,2011248_20161215__L_c2.jpg,NaN,2011248_20161215__L_c3.jpg,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lc


In [2]:
# make a list of the unique source training images that has 'training' = 0
# each image path consists of image directory, image name

df_train = df[df['training'] == 0]

# create a new df consists of image directory and image name concatenated
df_train['image_path'] = df_train['Directory'] + '/' + df_train['source']
df_train.head()


/tmp/ipykernel_2286792/1623562252.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['image_path'] = df_train['Directory'] + '/' + df_train['source']


,source,Source ROI,target,Target ROI,training,Warped target images,Warped target ROI,Execution time,Directory,image_path
3,2011248_20161215__L_c2.jpg,NaN,2011248_20161215__L_c1.jpg,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lc,Dataset/Dataset-processed/15-12-2559/2011248/L...
4,2011248_20161215__L_c2.jpg,NaN,2011248_20161215__L_c3.jpg,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lc,Dataset/Dataset-processed/15-12-2559/2011248/L...
5,2011248_20161215__L_c1.jpg,NaN,2011248_20161215__L_c3.jpg,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Lc,Dataset/Dataset-processed/15-12-2559/2011248/L...
6,2011248_20161215__R_b2.jpg,NaN,2011248_20161215__R_b3.jpg,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Rb,Dataset/Dataset-processed/15-12-2559/2011248/R...
7,2011248_20161215__R_b2.jpg,NaN,2011248_20161215__R_b1.jpg,NaN,0,NaN,NaN,NaN,Dataset/Dataset-processed/15-12-2559/2011248/Rb,Dataset/Dataset-processed/15-12-2559/2011248/R...


In [3]:
print(df_train.shape)
# make a list of the unique values in the column 'image_path'
image_list = df_train['image_path'].unique()[:500]
print(len(image_list))
print(image_list[0])


(4132, 10)
500
Dataset/Dataset-processed/15-12-2559/2011248/Lc/2011248_20161215__L_c2.jpg


In [4]:
min_translation = 0.1  # Minimum translation in pixels
max_translation = 0.2  # Maximum translation in pixels
max_rotation = 10  # Maximum rotation in degrees
max_shear = 10  # Maximum shear in degrees
min_scale = 0.85  # Minimum scale factor
max_scale = 1.15  # Maximum scale factor


# Apply only scaling to the eye images

In [5]:
def generate_affine_transformed_images_multiple(img_list, csv_file, output_dir, num_images=5, modify=False):
    # delete all files and subdirectories in the output directory
    shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    # create a list to store different point locations
    points_list = []

    # Initialize the CSV file with a header
    with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["source", "target", "M00", "M01", "M02", "M10", "M11", "M12", "image_path", "keypoints"])

    # Loop over the images, read the image, 
    # apply affine transformation and save it
    for i, img_path in enumerate(img_list):
        # if i > len(img_list)/2:
        #     break
        # Read the image as grayscale using cv2
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Save original image
        original_image_path = os.path.join(output_dir, f"img_{i}_original.png")

        # take 90% of the image
        image_base = image[int(image.shape[0]*0.05):int(image.shape[0]*0.95), int(image.shape[1]*0.05):int(image.shape[1]*0.95)]
        # resize image to 256x256
        image_base = cv2.resize(image_base, (256, 256))

        cv2.imwrite(original_image_path, image_base + np.random.uniform(-0.01, 0.01, image_base.shape))

        # Convert the transformed image to a numpy array
        # img_transformed = np.array(img_transformed)
        image_base = np.array(Image.fromarray(image_base).convert('L'))

        tracker = PointTracker(5, nn_thresh=0.7)
        points1, desc1, _ = superpoint(image_base.astype(np.float32)/255)

        for j in range(num_images):
            # random (2x3) affine transformation matrix
            #M = np.array([[1.0, 0.0, np.random()], [0.0, 1.0, 0.0]])
            if j == num_images-1:
                img_transformed = image_base
                M = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0]])
                points2 = points1.copy()
                desc2 = desc1.copy()
                # matches = tracker.nn_match_two_way(desc1, desc2, nn_thresh=0.7)
                # matches1 = points1[:2, matches[0, :].astype(int)]
                # matches2 = matches1
                # matches1_transformed_DVF = matches1
                # points_list.append(matches1_transformed_DVF[:, 0] - matches2[:, 0])

                # random (2x3) affine transformation matrix
                # M = np.array([[1.0, 0.0, np.random()], [0.0, 1.0, 0.0]])
                # rand_range = 0.2
                # test_random = np.random.uniform(-rand_range, rand_range, 2)
                # M = np.array([[1.0, 0.0, test_random[0]], [0.0, 1.0, test_random[1]]])

                # random (2x3) affine transformation matrix
                # random only translation and scale
                # M = np.array([[1.0 + np.random.uniform(-0.2, 0.2), 0.0, np.random.uniform(-0.2, 0.2)], 
                #               [0.0, 1.0 + np.random.uniform(-0.2, 0.2), np.random.uniform(-0.2, 0.2)]])
            
            else:
                rand_range = 0.1
                test_random = [np.random.uniform(-3*rand_range, rand_range), np.random.uniform(-1.5*rand_range, rand_range)]
                # M = np.array([[1.0 + test_random[0], 0.0, 0.0], [0.0, 1.0 + test_random[1], 0.0]])
                M = np.array([[1.0 + test_random[0], 0.0, 0.0], [0.0, 1.0 + test_random[1], 0.0]])

                # calculate the pixel coverage of the translation
                # print(f'Img {i}, translation: {test_random[0]*image.shape[1]}, {test_random[1]*image.shape[0]}')

                # image pixel shifted from image_base TODO: the random translation must not exceed the image_base
                # x_shift_1 = int(image.shape[1]*0.05) + int(test_random[0]*image.shape[1])
                # x_shift_2 = int(image.shape[1]*0.95) + int(test_random[0]*image.shape[1])
                # y_shift_1 = int(image.shape[0]*0.05) + int(test_random[1]*image.shape[0])
                # y_shift_2 = int(image.shape[0]*0.95) + int(test_random[1]*image.shape[0])
                # img_transformed = image[y_shift_1:y_shift_2, x_shift_1:x_shift_2]
                # print(img_transformed.shape)

                img_transformed = tensor_affine_transform(torch.tensor(image).unsqueeze(0).unsqueeze(0).float(), torch.tensor(M).unsqueeze(0).float())
                img_transformed = img_transformed.squeeze(0).squeeze(0).numpy()
                img_transformed = img_transformed[int(image.shape[0]*0.05):int(image.shape[0]*0.95), int(image.shape[1]*0.05):int(image.shape[1]*0.95)]

            # resize image to 256x256
            img_transformed = cv2.resize(img_transformed, (256, 256))
            # convert to grayscale
            img_transformed_BW = np.array(Image.fromarray(img_transformed).convert('L'))

            # # TODO: save heatmaps for other version of network
            points2, desc2, _ = superpoint(img_transformed_BW.astype(np.float32)/255)
            matches = tracker.nn_match_two_way(desc1, desc2, nn_thresh=0.7)

            # # print(desc1.shape, desc2.shape)
            matches1 = points1[:2, matches[0, :].astype(int)]
            # # matches1 = matches1.T[None, :, :]
            matches2 = points2[:2, matches[1, :].astype(int)]
            
            # transform the points using the displacement field
            # print(torch.tensor(M)[None, :, :].shape, torch.tensor(image)[None, None, :, :].shape)
            # print(torch.tensor(M).shape, torch.tensor(image).shape, torch.tensor(matches1.copy()).unsqueeze(-1).view(2, -1, 1).shape)
            matches1_transformed_DVF = transform_points_DVF(torch.tensor(matches1.copy()).unsqueeze(-1).view(2, -1, 1), 
                torch.tensor(M).view(1, 2, 3), torch.tensor(image_base).unsqueeze(0).unsqueeze(0))
            # print(f'Img {i}, diff: {matches1_transformed_DVF[:, 0] - matches2[:, 0]}')
            # points_list.append(matches1_transformed_DVF[:, 0] - matches2[:, 0])

            # add some noise to the transformed image and save it
            img_transformed = img_transformed + np.random.uniform(-0.01, 0.01, img_transformed.shape)
            if modify: # if modify is True, then add some intensity change to the transformed image
                img_transformed = img_transformed + np.random.normal(1, 0.1, 1)


            transformed_image_path = os.path.join(output_dir, f"img_{i}_transformed_{j}.png")
            cv2.imwrite(transformed_image_path, img_transformed)

            # create a dataframe with the matches
            # print(matches1.shape, matches2.shape, matches1_transformed_DVF.shape)
            if len(matches1_transformed_DVF.shape) == 3:
                matches1_transformed_DVF = matches1_transformed_DVF.squeeze(-1)
            df = pd.DataFrame({'x1': matches1[0, :], 'y1': matches1[1, :],
                            'x2': matches2[0, :], 'y2': matches2[1, :],
                            'x2_': matches1_transformed_DVF[0, :], 'y2_': matches1_transformed_DVF[1, :]})
            save_name = os.path.join(output_dir, f"img_{i}_{j}_keypoints.csv")
            df.to_csv(save_name, index=False)

            with open(csv_file, 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow([original_image_path, transformed_image_path, 
                                M[0, 0], M[0, 1], M[0, 2], M[1, 0], M[1, 1], M[1, 2], img_path, save_name])


    print(f"\nGenerated {(i + 1)*num_images} images")
    # print mean absolute error of the points
    # print('MAE point location error:', np.mean(np.abs(np.array(points_list))))



# Train data generate

In [6]:
# Define parameters
output_dir = "Dataset/synth_eye_medium_train_scaling"  # Output directory
os.makedirs(output_dir, exist_ok=True)

# plot_dir = "Dataset/synthetic_eye_dataset_train/plot"
# os.makedirs(plot_dir, exist_ok=True)

# generate synthetic images for each source training image
generate_affine_transformed_images_multiple(image_list,
    'Dataset/synth_eye_medium_train_scaling.csv', output_dir, num_images=20, modify=False)

/home/pakpoom/codes/spppt-2/.venv/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]



Generated 10000 images


In [7]:
# # Define parameters
# image_size = (512, 512)  # Size of the images
# output_dir = "Dataset/synthetic_eye_dataset_train_multiple"  # Output directory
# os.makedirs(output_dir, exist_ok=True)

# # plot_dir = "Dataset/synthetic_eye_dataset_train_multiple/plot"
# # os.makedirs(plot_dir, exist_ok=True)


# # generate synthetic images for each source training image
# generate_affine_transformed_images_multiple(image_list,'dataset_eye_synth_train_multiple.csv', output_dir, num_images=2, modify=True)

# Test data generate

In [7]:
# do the same for the test images
df_test = df[df['training'] == 1]
df_test['image_path'] = df_test['Directory'] + '/' + df_test['source']
print(df_test.shape)
image_list_test = df_test['image_path'].unique()[:100]

# Define parameters
output_dir = "Dataset/synth_eye_medium_test"  # Output directory
os.makedirs(output_dir, exist_ok=True)

# plot_dir = "Dataset/synthetic_eye_dataset_test/plot"
# os.makedirs(plot_dir, exist_ok=True)

# generate synthetic images for each source test image
generate_affine_transformed_images_multiple(image_list_test, 'Dataset/synth_eye_medium_test.csv', 
                                            output_dir, num_images=2)



/tmp/ipykernel_2286792/2538931727.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['image_path'] = df_test['Directory'] + '/' + df_test['source']


(908, 10)

Generated 200 images


In [24]:
# # Define parameters
# image_size = (512, 512)  # Size of the images
# output_dir = "Dataset/synthetic_eye_dataset_test_multiple"  # Output directory
# os.makedirs(output_dir, exist_ok=True)

# # plot_dir = "Dataset/synthetic_eye_dataset_test/plot"
# # os.makedirs(plot_dir, exist_ok=True)

# # generate synthetic images for each source test image
# generate_affine_transformed_images_multiple(image_list_test, 'dataset_eye_synth_test_scaling.csv', output_dir, num_images=1)